In [1]:
import tqdm
import multiprocessing
from multiprocessing import Pool
from joblib import Parallel, delayed
import pandas as pd
import sys

# manager = multiprocessing.Manager()
# projectlist = manager.list()
# ttable=manager.list()
big_table = pd.DataFrame()

In [2]:
#fnx and fnc are file path for final output
import os
load_path = os.getcwd()
fnx=load_path + '/output44/2018_COP_phase_1_result.xlsx'
fnc=load_path + '/output44/2018_COP_phase_1_result.csv'


#Import all the necessary modules and packages

# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
# from pdfminer.converter import TextConverter, XMLConverter, HTMLConverter
# from pdfminer.layout import LAParams
# from pdfminer.pdfpage import PDFPage
# from io import BytesIO
# import re
# import importlib
# importlib.reload(sys)
# sys.setdefaultencoding('utf8')
# import nltk
# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.sentiment.vader import SentimentIntensityAnalyzer
# from nltk.corpus import stopwords
# from nltk.stem.wordnet import WordNetLemmatizer
# from google.cloud import logging
# from datetime import timedelta
# from itertools import product
# import pandas_market_calendars as mcal
# from datetime import date,timedelta,datetime
# from google.cloud import translate
# from google.cloud import bigquery
# from google.cloud.bigquery import SchemaField,client
# from google.cloud.bigquery import client
# from newspaper import Article
# from newspaper import fulltext
# import xlsxwriter
# from textblob import TextBlob
# from openpyxl import load_workbook
# from gensim import corpora, models
# from gensim.summarization import summarize
# import gensim
# Gensim
# import gensim
# import gensim.corpora as corpora
# from gensim.utils import simple_preprocess
# from gensim.models import CoherenceModel
#
# # spacy for lemmatization
# import spacy
#
# # Plotting tools
# import pyLDAvis
# import pyLDAvis.gensim  # don't skip this
# import matplotlib.pyplot as plt
# %matplotlib inline
# from google.cloud import language
# from google.cloud.language import enums
# from google.cloud.language import types
# from sumy.parsers.plaintext import PlaintextParser #We're choosing a plaintext parser here, other parsers available for HTML etc.
# from sumy.nlp.tokenizers import Tokenizer
# from sumy.summarizers.lsa import LsaSummarizer as Summarizer
# from sumy.nlp.stemmers import Stemmer
# from sumy.utils import get_stop_words
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
# from sklearn.feature_extraction.text import TfidfVectorizer
# import collections
# import unicodedata
# import string
# import argparse
# from nltk.tokenize import RegexpTokenizer
# from stop_words import get_stop_words
# from nltk.stem.porter import PorterStemmer
# import datetime as dt
# # Enable logging for gensim - optional
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)
# from openpyxl import load_workbook

import time
import sys
import warnings
import re
import json

import img2pdf
from PIL import Image
import textract
import PyPDF2
from langdetect import detect
import time
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import textwrap
import requests
import uuid
import glob
#### Time out
import signal
from time import sleep    # only needed for testing
import docx
import pikepdf

#generate a new function called getpagenum, the input is the directory of an article
#the output is the number of page of each article
# def getpagenum(path):
#                 pdf = PyPDF2.PdfFileReader(open(path, "rb"),strict=False)
#                 num_of_pages = pdf.getNumPages()
#                 return num_of_pages

# Custom exception for the timeout
class TimeoutException(Exception):
    pass

# Handler function to be called when SIGALRM is received
def sigalrm_handler(signum, frame):
    # We get signal!
    raise TimeoutException()

##### Convert Docx and PPTX: PPTX returns a list
from pptx import Presentation
# path_to_presentation=directory+'/'+str('451161')+'/original/'+ 'Pacto_Global_-_COP_2016-17.pptx'
def getPPT(path_to_presentation):
    prs = Presentation(path_to_presentation)

    # text_runs will be populated with a list of strings,
    # one for each text run in presentation
    text_runs = []
    slide_num = 1

    for slide in prs.slides:
        for shape in slide.shapes:
            if not shape.has_text_frame:
                continue
            for paragraph in shape.text_frame.paragraphs:
                for run in paragraph.runs:
                    text_runs.append(run.text)
        slide_num += 1
    return [slide_num, text_runs]

def getText(filename):
    doc = docx.Document(filename)
    fullText = []
    for para in doc.paragraphs:
        fullText.append(para.text)
    return '\n'.join(fullText)
'''
def get_translated_text(text):
    body = [{'text': text}]
    request = requests.post(constructed_url, headers=headers, json=body)
    response = request.json()
    #return response
    return response[0]["translations"][0]["text"]

def reTrans(df):
    #This is to check if we failed to translate any row in df.
    #If there is, we try to translate them agin
    mask = df['translation'] == 'Fail to translate'
    def reTrans_helper(txt):
        if len(txt)>=unit_limit:
            #raw_str = temp_str
            print(str(i),df_result.corp_id[i],file_name,'too big to translate')
            return 'Too big'
        else:
            try:
                if len(txt) <= 4000:
                    return get_translated_text(txt)
                else:
                    lines = textwrap.wrap(txt, 4000, break_long_words=False)
                    #translate_client = translate.Client()
                    return ''.join([get_translated_text(x) for x in lines])
                print(str(i),df_result.corp_id[i],file_name,'Translated')
            except Exception as e:
                #raw_str = temp_str
                print(str(i),df_result.corp_id[i],file_name,'Fail to translate')
                print("ERROR MESSAGE reTrans: ")
                print(e)
                return 'Fail to translate'

    df.loc[mask, 'translation'] = df.loc[mask, 'text'].apply(reTrans_helper)
    #df['translation'] = df['text'].apply(lambda text: get_translated_text(text) if )
'''

def pdf2text_and_numpage(filename):
    #write a for-loop to open many files
    #open allows you to read the file
    pdfFileObj = open(filename,'rb')
#The pdfReader variable is a readable object that will be parsed
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
#discerning the number of pages will allow us to parse through all #the pages
    num_pages = pdfReader.numPages
    count = 0
    text = ""
#The while loop will read each page
    while count < num_pages:
        pageObj = pdfReader.getPage(count)
        count +=1
        text += pageObj.extractText()

    #This if statement exists to check if the above library returned #words. It's done because PyPDF2 cannot read scanned files.
    if text != "":
        if '\n\n\n\n\n\n\n\n\n\n' in text:
            return (str(textract.process(filename, method='tesseract')), num_pages, 'text-based-PDF')
        else:
            return (str(text), num_pages, 'text-based-PDF')
    #If the above returns as False, we run the OCR library textract to #convert scanned/image based PDF files into text
    else:
        return (str(textract.process(filename, method='tesseract')), num_pages, 'image-based-PDF')
    # Now we have a text variable which contains all the text derived #from our PDF file. Type print(text) to see what it contains. It #likely contains a lot of spaces, possibly junk such as '\n' etc.
    # Now, we will clean our text variable, and return it as a list of keywords.

def image2text_and_numpage(img_path, path2original):
    # storing pdf path
    pdf_path = path2original + 'converted.pdf'
    # opening image
    image = Image.open(img_path)
    # converting into chunks using img2pdf
    pdf_bytes = img2pdf.convert(image.filename)
    # opening or creating pdf file
    file = open(pdf_path, "wb")
    # writing pdf files with chunks
    file.write(pdf_bytes)
    # closing image file
    image.close()
    # closing pdf file
    file.close()
    return pdf2text_and_numpage(pdf_path)

def decrypt_code(filename, path2original):
    if PyPDF2.PdfFileReader(open(filename, 'rb')).isEncrypted:
        pdf_old = pikepdf.open(filename)
        pdf_path = path2original+'extractable.pdf'
        pdf_old.save(pdf_path)
        return pdf2text_and_numpage(pdf_path)
    # skip file? Write to a log?
    else:
        return pdf2text_and_numpage(filename)

'''
def translation_clean(text):
    cidcompile1=re.compile(r'\(*\s*\(*\s*[cC]\s*i\s*\)*\s*d\s*\:*\s*\:*(?:\(*\s*[cC]\s*i\s*d\s*\:*)?\:*\s*[0-9]+\s*[0-9]{0,}\s*\)*\s*\)*')
    cidcompile2=re.compile(r'\(\s*[cC]\s*i\s*\)*\s*d\s*\:')
    cidcompile3=re.compile(r'\:\s*[0-9]+\s*\)')
    punctuation = re.compile(r',.:?!<>{};')
    
    t_str=re.sub(cidcompile1,' ',text)
    t_str=re.sub(cidcompile2,' ',t_str)
    t_str=re.sub(cidcompile3,' ',t_str)
    t_str=re.sub(punctuation,' ',t_str)
    t_str=re.sub("\n",' ',t_str)
    
    control = re.compile('\x00|\x01|\x02|\x03|\x04|\x05|\x06|\x07|\x08|\x09|\x0a|\x0b|\x0c|\x0d|\x0e|\x0f|\x10|\x11|\x12|\x13|\x14|\x15|\x16|\x17|\x18|\x19|\x1a|\x1b|\x1c|\x1d|\x1e|\x1f|\x7f|\xc2\x80|\xc2\x81|\xc2\x82|\xc2\x83|\xc2\x84|\xc2\x85|\xc2\x86|\xc2\x87|\xc2\x88|\xc2\x89|\xc2\x8a|\xc2\x8b|\xc2\x8c|\xc2\x8d|\xc2\x8e|\xc2\x8f|\xc2\x90|\xc2\x91|\xc2\x92|\xc2\x93|\xc2\x94|\xc2\x95|\xc2\x96|\xc2\x97|\xc2\x98|\xc2\x99|\xc2\x9a|\xc2\x9b|\xc2\x9c|\xc2\x9d|\xc2\x9e|\xc2\x9f')
    
    t_str = re.sub(control,' ',t_str)
    
    return t_str
'''

## save all the folders name of articles into the corp_id list
directory = load_path + '/COP44/'
os.chdir(directory)


#print(os.getcwd())
corp_id = [x for x in glob.glob('*')]

#This step is to save all the name and id of articles in list1 and files_name list

#num is used to calculate the number of empty folders
num = 0
#a is used to calculate the number of non-empty folders
a = 0
#list1 is used to save all the id of non-empty articles
list1 = []
#files_name is a list used to save all the names of non-empty articles
files_name = []

for i in corp_id:
    ##each path is the path of each articles
    path = directory + '/' + i + '/original'
    for dirpath, dirnames, files in os.walk(path):
        if not files:
            #print(dirpath, 'is empty')
            num += 1
        else:
            a += 1
            #print(dirpath,'is not empty')
            list1.append(i)
            files_name.append(files)

#generate a dataframe which includes the corporate_id and size
#d=pd.DataFrame(columns=['corpor_id','file_size_in_bytes'])
#result_list is a list of list, which contains all id, file names and size of all the elements of articles
result_list = []
fail_file = []
for i in range(0, len(files_name)):
    if len(files_name[i]) == 1:
        ## file_name is the first element of each item in files_name list
        file_name = str(files_name[i][0])
        try:
            #### list1 is id
            path1 = directory + '/' + str(list1[i]) + '/original/' + file_name
            ## statinfo is the size of each file Size in bytes of a plain file; amount of data waiting on some special files.
            statinfo = os.stat(path1).st_size
            result_list.append([list1[i],file_name,statinfo])
        except:
            fail_file.append([i,list1[i],file_name])
            pass
    else:
        for l in range(0,len(files_name[i])):
            file_name = str(files_name[i][l])
            try:
                if "desktop" in file_name:
                    continue
                else:
                    path1 = directory + '/' + str(list1[i]) + '/original/' + file_name
                    statinfo = os.stat(path1).st_size
                    result_list.append([list1[i], file_name, statinfo])
            except:
                fail_file.append([i,list1[i],file_name])
                pass

labels = ['corp_id', 'file_name', 'file_size_in_bytes']
#generate a dataframe called df_result with column names 'corp_id','file_name','size'
#the elements in df_result is result_list
df_result = pd.DataFrame(result_list, columns=labels)

####LIMIT FOR CLOUD JSON IS DIFFERENT
unit_limit= 10000000
#os.environ["GOOGLE_APPLICATION_CREDENTIALS"]='/home/globalaicloud/UNGC_NEW/JSON/My First Project-dec55926c2d0.json'

# Checks to see if the Translator Text subscription key is available
# as an environment variable. If you are setting your subscription key as a
# string, then comment these lines out.
if 'TRANSLATOR_TEXT_KEY' in os.environ:
    subscriptionKey = os.environ['TRANSLATOR_TEXT_KEY']
else:
    print('Environment variable for TRANSLATOR_TEXT_KEY is not set.')
    #exit()
# If you want to set your subscription key as a string, uncomment the line
# below and add your subscription key.
subscriptionKey = "331f7ace25a849639d0d319181758dff"

base_url = 'https://api.cognitive.microsofttranslator.com'
path = '/translate?api-version=3.0'
params = '&to=en'
constructed_url = base_url + path + params

headers = {
    'Ocp-Apim-Subscription-Key': subscriptionKey,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())}

timelimit_seconds = 60*30    # Must be an integer


### fill the columns in each row
###"file format" "page number" "original text" ""

def extract_text(i):
    path1 = directory + '/' + str(df_result.corp_id[i]) + '/original/' + df_result.file_name[i]
    path2original = directory + '/' + str(df_result.corp_id[i]) + '/original/'
    file_name = str(df_result.file_name[i])
    if file_name.endswith('.doc') or file_name.endswith('.docx'):
        df_result.loc[i, 'file_format'] = 'Word File'
        df_result.loc[i, 'page_num'] = np.NaN
        try:
            df_result.loc[i, 'text'] = getText(path1)
            print(str(i), df_result.corp_id[i], file_name, 'Doc converted')
        except:
            df_result.loc[i, 'text'] = 'Wrong Format'
            print(str(i), df_result.corp_id[i], file_name, 'Doc unconverted')
    elif file_name.endswith('.ppt') or file_name.endswith('.pptx'):
        df_result.loc[i, 'file_format'] = 'Powerpoint'
        try:
            ppt_text = getPPT(path1)
            df_result.loc[i, 'page_num'] = ppt_text[0]
            df_result.loc[i, 'text'] = ''.join(x for x in ppt_text[1])
            print(str(i), df_result.corp_id[i], file_name, 'PPT converted')
        except:
            df_result.loc[i, 'page_num'] = np.NaN
            df_result.loc[i, 'text'] = 'Wrong Format'
            print(str(i), df_result.corp_id[i], file_name, 'PPT unconverted')
    elif file_name.endswith('.jpg') or file_name.endswith('.jpeg') or file_name.endswith('.png'):
        try:
            wildcard, file_extension = os.path.splitext(path1)
            image_text, num_page, wildcard = image2text_and_numpage(path1, path2original)
            df_result.loc[i, 'page_num'] = num_page
            df_result.loc[i, 'text'] = image_text
            df_result.loc[i, 'file_format'] = file_extension[1:]
            print(str(i), df_result.corp_id[i], file_name, 'image converted')
        #         df_result.loc[i,'file_format']='Image'
        #         df_result.loc[i,'page_num']=np.NaN
        #         df_result.loc[i,'text']='Wrong Format'
        #         print(str(i),df_result.corp_id[i],file_name, 'Wrong Format')
        except Exception as e:
            print("Error Message for IMAGE READ: ")
            print(e)
    elif file_name.endswith('.xlsx') or file_name.endswith('.csv'):
        df_result.loc[i, 'file_format'] = 'Excel'
        df_result.loc[i, 'page_num'] = np.NaN
        df_result.loc[i, 'text'] = 'Wrong Format'
        print(str(i), df_result.corp_id[i], file_name, 'Wrong Format')
    else:
        # df_result.loc[i,'file_format']='pdf'
        
        raw_str = ''
        
        # Set up signal handler for SIGALRM, saving previous value
        
        old_handler = signal.signal(signal.SIGALRM, sigalrm_handler)
        
        # Start timer
        
        signal.alarm(timelimit_seconds)
        
        try:
            # raw = parser.from_file('sample.pdf')
            # print(raw['content'])
            # pdf_text = tika.parser.from_file(path1)
            pdf_text, num_page, pdf_type = decrypt_code(path1, path2original)
            df_result.loc[i, 'page_num'] = num_page
            df_result.loc[i, 'text'] = pdf_text
            df_result.loc[i, 'file_format'] = pdf_type
            print(str(i), df_result.corp_id[i], file_name, 'PDF converted')
            # print('PDF TEXT:')
            # print(pdf_text[1])
        except Exception as e:
            if str(e) is "":
                df_result.loc[i, 'text'] = 'Timeout'
                df_result.loc[i, 'language'] = 'Timeout'
                df_result.loc[i, 'page_num'] = np.NaN  
                print(str(i), df_result.corp_id[i], file_name, 'took too long to convert')
            else:
                print('ERROR MESSAGE PDF:')
                print(e)
                df_result.loc[i, 'text'] = 'Convert Manually'
                df_result.loc[i, 'page_num'] = np.NaN
                df_result.loc[i, 'language'] = 'Error'
                print(str(i), df_result.corp_id[i], file_name, 'fail to convert')
        finally:
            # Turn off timer
            signal.alarm(0)
            # Restore handler to previous value
            signal.signal(signal.SIGALRM, old_handler)

    #count the word_count
    txt = df_result.loc[i, 'text']
#    txt = translation_clean(txt)
    if txt != "Actually image":
        word_count = len(txt.split())
    else:
        word_count = -1
    df_result.loc[i,'word_count'] = word_count

    try:
        # print('trytrytrytrytrytrytrytry')
        # print('df_result.loc[i,text]:')
        # print(type(df_result.loc[i,'text']))
        lang = detect(df_result.loc[i, 'text'])
        if lang == 'en':
            # print('ifififififififif')
            df_result.loc[i, 'language'] = 'English'
            print(str(i), df_result.corp_id[i], file_name, 'English')
        else:
            df_result.loc[i, 'language'] = lang
            # print('elselseelseelseelse')
            # if len(df_result.loc[i, 'text']) >= unit_limit:
            #     # raw_str = temp_str
            #     df_result.loc[i, 'translation'] = 'Too big'
            #     print(str(i), df_result.corp_id[i], file_name, 'too big to translate')
            # else:
            #     try:
            #         if len(df_result.loc[i, 'text']) <= 4000:
            #             df_result.loc[i, 'translation'] = get_translated_text(df_result.loc[i, 'text'])
            #         else:
            #             lines = textwrap.wrap(df_result.loc[i, 'text'], 4000, break_long_words=False)
            #             # translate_client = translate.Client()
            #             df_result.loc[i, 'translation'] = ''.join([get_translated_text(x) for x in lines])
            #         print(str(i), df_result.corp_id[i], file_name, 'Translated')
            #     except Exception as e:
            #         # raw_str = temp_str
            #         df_result.loc[i, 'translation'] = 'Fail to translate'
            #         print(str(i), df_result.corp_id[i], file_name, 'Fail to translate')
            #         print("ERROR MESSAGE translation: ")
            #         print(e)
    except Exception as e:
        print('ERROR MESSAGE detect:')
        print(e)
        df_result.loc[i, 'text'] = 'No content'
        df_result.loc[i, 'page_num'] = np.NaN
        df_result.loc[i, 'language'] = 'No content'
        print(str(i), df_result.corp_id[i], file_name, 'No content')
        
    time.sleep(1)
        
#    ttable.append(df_result.iloc[[i]])
    return df_result.iloc[[i]]
    

        
        
        
#     writer = pd.ExcelWriter(fnx)
#     df_result.loc[:, :].to_excel(writer, 'sheet1', header=True, index=True)
#     writer.save()
#     writer.close()
#     df_result.loc[:, :].to_csv(fnc, index=True)
#     print(str(i), df_result.corp_id[i], ' finish step', fnc)

#     print("Finish textracting " + df_result.corp_id[i] + " !")



# This is to check if we failed to translate any text in df_result.
# If there is, we try to translate those text agin
'''
reTrans(df_result)
'''


Environment variable for TRANSLATOR_TEXT_KEY is not set.


'\nreTrans(df_result)\n'

In [3]:
result_list[1]

['464159', 'COP_Chauveau_2017.pdf', 16948813]

In [ ]:
list_result = range(len(result_list))
start=time.time()
num_cores=multiprocessing.cpu_count()
final_pd = Parallel(n_jobs=num_cores)(delayed(extract_text)(k) for k in tqdm.tqdm_notebook(list_result))

print('------time used: {} ------------'.format(time.time()-start))

for i in final_pd:
    big_table=pd.concat([big_table,i],axis=0,sort=False)

writer = pd.ExcelWriter(fnx)
big_table.loc[:, :].to_excel(writer, 'sheet1', header=True, index=True)
writer.save()
writer.close()
big_table.loc[:, :].to_csv(fnc, index=True)

spreads = pd.read_csv(load_path + '/output44/2018_COP_phase_1_result.csv')
spreads1 = spreads.loc[spreads['language'] == 'English']
fnd=load_path + '/output44/2018_COP_phase_1_result_English.xlsx'
fnf=load_path + '/output44/2018_COP_phase_1_result_English.csv'


writer = pd.ExcelWriter(fnd)
spreads1.loc[:, :].to_excel(writer, 'sheet1', header=True, index=True)
writer.save()
writer.close()
spreads1.loc[:, :].to_csv(fnf, index=True)

In [ ]:
for i in final_pd:
    big_table=pd.concat([big_table,i],axis=0,sort=False)

writer = pd.ExcelWriter(fnx)
big_table.loc[:, :].to_excel(writer, 'sheet1', header=True, index=True)
writer.save()
writer.close()
big_table.loc[:, :].to_csv(fnc, index=True)

spreads = pd.read_csv(load_path + '/output2/2018_COP_phase_1_result.csv')
spreads1 = spreads.loc[spreads['language'] == 'English']
fnd=load_path + '/output2/2018_COP_phase_1_result_English.xlsx'
fnf=load_path + '/output2/2018_COP_phase_1_result_English.csv'


writer = pd.ExcelWriter(fnd)
spreads1.loc[:, :].to_excel(writer, 'sheet1', header=True, index=True)
writer.save()
writer.close()
spreads1.loc[:, :].to_csv(fnf, index=True)

In [ ]:
for i in final_pd:
    big_table=pd.concat([big_table,i],axis=0,sort=False)

In [ ]:
big_table

In [ ]:
writer = pd.ExcelWriter(fnx)
big_table.loc[:, :].to_excel(writer, 'sheet1', header=True, index=True)
writer.save()
writer.close()
big_table.loc[:, :].to_csv(fnc, index=True)

In [ ]:
spreads = pd.read_csv(load_path + '/output2/2018_COP_phase_1_result.csv')
spreads1 = spreads.loc[spreads['language'] == 'English']
fnd=load_path + '/output2/2018_COP_phase_1_result_English.xlsx'
fnf=load_path + '/output2/2018_COP_phase_1_result_English.csv'


writer = pd.ExcelWriter(fnd)
spreads1.loc[:, :].to_excel(writer, 'sheet1', header=True, index=True)
writer.save()
writer.close()
spreads1.loc[:, :].to_csv(fnf, index=True)
